How to set up selenium and client?

In [ ]:
var importer = require('../Core');

var client, results, getCredentials, loginGoogle, getAllXPath;
var getClient = () => {
    return importer.interpretAll([
        'webdriver client',
        'decrypt password',
        'log in Google',
        'get all elements'
    ])
    .then(r => {
        results = r;
        return r[0].runInNewContext();
    })
    .then(r => {
        client = r();
        getCredentials = results[1].runInNewContext({client});
        loginGoogle = results[2].runInNewContext({client, getCredentials});
        getAllXPath = results[3].runInNewContext({client});
        return client;
    });
};
module.exports = getClient;
getClient;



In [ ]:
$$.async();
getClient()
    .then(c => $$.sendResult(c))
    .catch(e => $$.sendError(e));


How to download google timeline data using webdriver?

In [ ]:
var getGoogleTimeline = () => {
    console.log('Google: Logging timeline history');
    return client
    .url('https://www.google.com/maps/timeline')
    .loginGoogle()
    .waitUntil(() => client.getUrl()
               .then(url => url.indexOf('timeline') > -1, 20000, ''))
    .catch(function (e) {
        console.log(e);
    })
    .pause(3000)
    .click('button[jsaction="header.select-today"]')
    .pause(2000);
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('getGoogleTimeline', getGoogleTimeline);
}
module.exports = getGoogleTimeline;
getGoogleTimeline;


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = '/Users/briancullinan/Timeline';

var readAllPages = () => {
    // TODO: add stop conditions
    var oneDay = 24*60*60*1000; // hours*minutes*seconds*milliseconds
    var firstDate = new Date(2016,8,21);
    var secondDate = new Date();
    var days = Math.round(
        Math.abs((firstDate.getTime() - secondDate.getTime())/(oneDay)));
    dayKey = '';
    var promises = [];
    for (var c = 0; c < days; c++) {
        var result;
        promises[promises.length] = (resolve) => readTimelinePage()
            .then(r => result = r)
            .click('.previous-date-range-button')
            .pause(2000)
            .then(() => {
                if(typeof(result) == 'undefined') {
                    resolve();
                }
                console.log(result);
                fs.writeFileSync(
                    PROJECT_PATH + '/timeline-' + result[0].timeline + '_.json',
                    JSON.stringify(result, null, 4))
                resolve(result);
            })
    }
    return promises;
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('readAllPages', readAllPages);
}
module.exports = readAllPages;
readAllPages;


In [ ]:
var chrono = require('chrono-node');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
var readTimelinePage = () => {
    return client.getAllXPath({
        day: '//*[contains(@class,"timeline-subtitle")]//text()',
        items: [
            '//*[contains(@class,"timeline-item")]/parent::*/*[@jsinstance]',
            {
                duration: './/*[contains(@class, "duration-text")]//text()',
                data: './/*[contains(@class, "timeline-item")]/@data-segment-key',
                title: './/*[contains(@class, "timeline-item-title-content")][.//i]/*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-title-content")][not(.//i)]//text()',
                location: './/*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][.//a]/a//*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][not(.//a)]/text()'
            }
        ]
    })
    .then(r => {
        console.log(r.day);
        if(typeof r.day != 'string') {
            return;
        }
        var currDate = chrono.parseDate(r.day);
        var newKey = currDate.getDate() 
            + months[currDate.getMonth()] 
            + (currDate.getFullYear() + '').substr(2, 2);
        if (newKey != dayKey) {
            dayKey = newKey;
        }
        
        return r.items.map(i => {
            var timelineData = (i.data + '').split(':');
            var start, end, length;
            if(timelineData[0]) {
                start = new Date(parseFloat(timelineData[1]));
                end = new Date(parseFloat(timelineData[2]));
                length = end.getTime() - start.getTime();
            } else {
                start = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[0]);
                end = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[1]);
                length = 0;
            }
            return ({
                type: 'timeline',
                timeline: dayKey,
                name: i.title,
                location: i.location,
                time: start,
                length: isNaN(length) ? 0 : length
            })
        })
    })
    .catch(e => console.log(e))
};
if (typeof client.readTimelinePage == 'undefined') {
    client.addCommand('readTimelinePage', readTimelinePage);
}
module.exports = readTimelinePage;
readTimelinePage;


In [ ]:
var fs = require('fs');
var importer = require('../Core');

$$.async()
client.getGoogleTimeline()
    .then((r) => importer.runAllPromises(readAllPages()))
    .catch(e => $$.sendError(e));



Load location export data?



In [ ]:
var importer = require('../Core');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
var dayKey;

var loadLocations = (file) => {
    var locationCache = {};
    return importer.streamJson(file, [true, {emitPath: true}], (match) => {
        if(match.path[0] === 'locations') {
            var currDate = new Date(parseInt(match.value.timestampMs));
            var newKey = currDate.getDate() + months[currDate.getMonth()]
                + (currDate.getFullYear() + '').substr(2, 2);
            if (newKey != dayKey) {
                dayKey = newKey;
            }
            var newRow = Object.assign({
                    time: currDate,
                    type: 'location',
                    location: dayKey,
            }, match.value);
            var cache = locationCache[newRow.location];
            if(typeof cache === 'undefined') {
                cache = locationCache[newRow.location] = [];
            }
            cache[cache.length] = newRow;
        }
    })
    .then(() => locationCache)
};
module.exports = loadLocations;
loadLocations;



Find the average latitute and longitude at each destination?



In [ ]:
var importer = require('../Core');
var _ = require('underscore');
var fs = require('fs');
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Timeline';

function toRadians (angle) {
  return angle * (Math.PI / 180);
}

var straightDistance = (lat1, lon1, lat2, lon2) => {
    var R = 6371e3; // metres
    var φ1 = toRadians(lat1);
    var φ2 = toRadians(lat2);
    var Δφ = toRadians(lat2-lat1);
    var Δλ = toRadians(lon2-lon1);

    var a = Math.sin(Δφ/2) * Math.sin(Δφ/2) +
            Math.cos(φ1) * Math.cos(φ2) *
            Math.sin(Δλ/2) * Math.sin(Δλ/2);
    var c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));

    var d = R * c;
    return d;
}

var averageDestinations = locationCache => {
    return importer.runAllPromises(Object.keys(locationCache).map(date => resolve => {
        console.log(date);
        if(!fs.existsSync(PROJECT_PATH + '/timeline-' + date + '_.json')) {
            return resolve();
        }
        var timelineData = JSON.parse(
            fs.readFileSync(PROJECT_PATH + '/timeline-' + date + '_.json')
            .toString());
        var destinations = [];
        for(const d of timelineData) {
            if((d.name + '') == '' || (d.name + '').indexOf('Driving') > -1) {
                continue;
            }
            const nearest = _.sortBy(
                locationCache[date],
                l => Math.abs(new Date(l.time).getTime() - new Date(d.time).getTime()))
                .slice(0, 3);
            // make sure it isn't off by much
            const averageLat = nearest.map(n => n.latitudeE7)
                .reduce((a, b) => a + b, 0) / nearest.length / 10000000;
            const averageLon = nearest.map(n => n.longitudeE7)
                .reduce((a, b) => a + b, 0) / nearest.length / 10000000;
            if(nearest.filter(nearby => straightDistance(
                    nearby.latitudeE7 / 10000000,
                    nearby.longitudeE7 / 10000000,
                    averageLat,
                    averageLon
                ) > 500).length > 0) {
                destinations.push(Object.assign(d, {
                    averageLat: averageLat,
                    averageLon: averageLon,
                    locations: nearest.map(nearby => Object.assign(nearby, {
                        averageLat: nearby.latitudeE7 - averageLat,
                        averageLon: nearby.longitudeE7 - averageLon,
                        averageDist: straightDistance(
                            nearby.latitudeE7 / 10000000,
                            nearby.longitudeE7 / 10000000,
                            averageLat,
                            averageLon
                        )
                    }))
                }));
            }
            destinations.push(Object.assign(d, {
                averageLat: averageLat,
                averageLon: averageLon
            }));
        }
        resolve([destinations]);
    }))
}
module.exports = averageDestinations;
averageDestinations;



Reconcile timeline data with calendar?



In [ ]:
var importer = require('../Core');

var getEvents = (time) => {
    const start = new Date(time);
    start.setHours(0, 0, 0);
    const end = new Date(time);
    end.setHours(23, 59, 0);
    return listEvents({
        auth: options.auth,
        timeMin: start.toString(),
        timeMax: end.toString()
    })
    .catch(e => console.log(e));
}

var filterEvents = (events, locations) => {
    const unmatched = [], matches = [];
    for(const l of locations) {
        const matching = events.filter(e => {
            return ((l.name + '').indexOf('Gainey') > -1
                && e.event.summary == 'Drive to work')
            || ((l.name + '').indexOf('Swiftpage') > -1
                && e.event.summary == 'Drive to work')
            || ((l.name + '').indexOf('6934') > -1
                && e.event.summary == 'Drive home')
            || ((l.name + '').indexOf('6934') > -1
                && e.event.summary == 'Work from home')
            || (typeof e.event.location !== 'undefined'
                && ((e.event.location + '').indexOf(l.location + '') > -1
                || (e.event.location + '').indexOf(l.name + '') > -1))
        });
        if(matching.length == 0) {
            unmatched.push(l);
        } else {
            matches.push(Object.assign(l, matching[0]));
        }
    }
    console.log('Unmatched ' + unmatched.length + ' -  out of ' + matches.length + ' - '
                + unmatched.map(u => u.name + '').join(', '));
}

var listEvents, options = {};
var nearbyCache = {};
var reconcileTimeline = (destinations) => {
    console.log(destinations[0].time);
    var placesNearby, getOauthClient, events, locations = [];
    return importer.interpretAll([
        'import google calendar api',
        'places nearby google maps',
        'list events',
    ])
    .then(r => {
        getOauthClient = r[0].runInNewContext();
        placesNearby = r[1].runInNewContext();
        listEvents = r[2].runInNewContext();
        return getOauthClient(options);
    })
    .then(() => getEvents(destinations[0].time))
    .then(r => {
        events = r;
        return importer.runAllPromises(destinations.map(d => resolve => {
            if(typeof nearbyCache[d.name + ', ' + d.location] !== 'undefined') {
                return resolve(nearbyCache[d.name + ', ' + d.location]);
            }
            return placesNearby(d.name + ', ' + d.location,
                            {lat: d.averageLat, lng: d.averageLon})
            .then(r => {
                if(r.length === 0) {
                    console.warn('No match for ' + JSON.stringify(d))
                    resolve();
                } else {
                    nearbyCache[d.name + ', ' + d.location] = r[0];
                    resolve(Object.assign(d, r[0]))
                }
            })
            .catch(e => resolve(e))
        }))
    })
    .then(r => {
        locations = r.filter(l => typeof l !== 'undefined');
        return filterEvents(events, locations);
    })
    .catch(e => console.log(e))
}
module.exports = reconcileTimeline;
reconcileTimeline;



In [1]:
var importer = require('../Core');

var loadLocations, averageDestinations, reconcileTimeline;

$$.async();
importer.interpretAll([
    'load locations export data',
    'average latitude longitude',
    'reconcile timeline calendar'
])
.then(r => {
    loadLocations = r[0].runInNewContext();
    averageDestinations = r[1].runInNewContext();
    reconcileTimeline = r[2].runInNewContext();
})
.then(() => {
    return loadLocations('/Users/briancullinan/Downloads/Takeout 7/Location History/Location History.json');
})
.then(locationCache => averageDestinations(locationCache))
.then(destinations => {
    return importer.runAllPromises(destinations
                                   .filter(d => typeof d !== 'undefined' && d.length > 0)
                                   .map(d => resolve => {
        return reconcileTimeline(d).then(() => resolve());
    }))
})
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e))



30Sep17
29Sep17
28Sep17
27Sep17
26Sep17
25Sep17
24Sep17
23Sep17
22Sep17
21Sep17
20Sep17
19Sep17
18Sep17
17Sep17
16Sep17
15Sep17
14Sep17
13Sep17
12Sep17
11Sep17
10Sep17
9Sep17
8Sep17
7Sep17
6Sep17
5Sep17
4Sep17
3Sep17
2Sep17
1Sep17
31Aug17
30Aug17
29Aug17
28Aug17
27Aug17
26Aug17
25Aug17
24Aug17
23Aug17
22Aug17
21Aug17
20Aug17
19Aug17
18Aug17
17Aug17
16Aug17
15Aug17
14Aug17
13Aug17
12Aug17
11Aug17
10Aug17
9Aug17
8Aug17
7Aug17
6Aug17
5Aug17
4Aug17
3Aug17
2Aug17
1Aug17
31Jul17
30Jul17
29Jul17
28Jul17
27Jul17
26Jul17
25Jul17
24Jul17
23Jul17
22Jul17
21Jul17
20Jul17
19Jul17
18Jul17
17Jul17
16Jul17
15Jul17
14Jul17
13Jul17
12Jul17
11Jul17
10Jul17
9Jul17
8Jul17
7Jul17
6Jul17
5Jul17
4Jul17
3Jul17
2Jul17
1Jul17
30Jun17
29Jun17
28Jun17
27Jun17
26Jun17
25Jun17
24Jun17
23Jun17
22Jun17
21Jun17
20Jun17
19Jun17
18Jun17
17Jun17
16Jun17
15Jun17
14Jun17
13Jun17
12Jun17
11Jun17
10Jun17
9Jun17
8Jun17
7Jun17
6Jun17
5Jun17
4Jun17
3Jun17
2Jun17
1Jun17
31May17
30May17
29May17
28May17
27May17
26May17
25May17
24Ma

No match for {"type":"timeline","timeline":"21Sep17","name":[" ","Moving"," - 6.9 mi"],"location":[],"time":"2017-09-21T22:06:36.038Z","length":920195,"averageLat":33.56594726666667,"averageLon":-111.91677,"locations":[{"time":"2017-09-21T22:06:36.038Z","type":"location","location":"21Sep17","timestampMs":"1506031596038","latitudeE7":335656717,"longitudeE7":-1119136229,"accuracy":98,"activity":[{"timestampMs":"1506031653768","activity":[{"type":"UNKNOWN","confidence":75},{"type":"ON_FOOT","confidence":10},{"type":"IN_VEHICLE","confidence":5},{"type":"ON_BICYCLE","confidence":5},{"type":"STILL","confidence":5},{"type":"WALKING","confidence":5},{"type":"RUNNING","confidence":5}]}],"averageLat":335656683.4340527,"averageLon":-1119136117.08323,"averageDist":293.19482658779685},{"time":"2017-09-21T22:08:37.156Z","type":"location","location":"21Sep17","timestampMs":"1506031717156","latitudeE7":335667250,"longitudeE7":-1119231785,"accuracy":116,"averageLat":335667216.4340527,"averageLon":-111

2017-09-21T14:00:00.000Z
Unmatched 4 -  out of 5 - 13802 N Scottsdale Rd, 13802 N Scottsdale Rd, Greenbrier East, Greenbrier East
2017-09-20T14:53:00.000Z
2017-09-20T15:00:00.000Z
Unmatched 0 -  out of 6 - 
2017-09-19T19:28:00.000Z
2017-09-19T00:00:00.000Z
Unmatched 0 -  out of 3 - 
2017-09-18T15:04:00.000Z
2017-09-18T15:00:00.000Z
Unmatched 0 -  out of 5 - 
2017-09-17T16:54:00.000Z
2017-09-17T00:00:00.000Z
Unmatched 0 -  out of 8 - 
2017-09-16T17:54:00.000Z
2017-09-16T00:00:00.000Z
Unmatched 2 -  out of 4 - Eileen Taggart | REMAX Fine Properties, Eileen Taggart | REMAX Fine Properties
2017-09-15T15:13:00.000Z
2017-09-15T15:00:00.000Z
Unmatched 0 -  out of 5 - 
2017-09-14T15:10:00.000Z


No match for {"type":"timeline","timeline":"14Sep17","name":[" ","Moving"," - 155 ft"],"location":[],"time":"2017-09-14T20:17:11.821Z","length":3204179,"averageLat":33.563727166666666,"averageLon":-111.91231696666668}


2017-09-14T13:00:00.000Z
Unmatched 0 -  out of 5 - 
2017-09-13T15:12:00.000Z
2017-09-13T15:00:00.000Z
Unmatched 0 -  out of 7 - 
2017-09-12T15:20:00.000Z
2017-09-12T15:00:00.000Z
Unmatched 0 -  out of 9 - 
2017-09-11T16:58:00.000Z
2017-09-11T15:00:00.000Z
Unmatched 0 -  out of 4 - 
2017-09-10T19:00:00.000Z
2017-09-10T00:00:00.000Z
Unmatched 1 -  out of 0 - 6934 E Sandra Terrace
2017-09-09T19:00:00.000Z
2017-09-09T00:00:00.000Z
Unmatched 1 -  out of 0 - 6934 E Sandra Terrace
2017-09-08T16:13:00.000Z
2017-09-08T15:30:00.000Z
Unmatched 2 -  out of 4 - McDonald's, McDonald's
2017-09-07T15:35:00.000Z
2017-09-07T15:00:00.000Z
Unmatched 0 -  out of 6 - 
2017-09-06T19:58:00.000Z
2017-09-06T16:00:00.000Z
Unmatched 2 -  out of 1 - 6934 E Sandra Terrace, 6934 E Sandra Terrace
2017-09-05T15:21:00.000Z
2017-09-05T15:00:00.000Z
Unmatched 0 -  out of 5 - 
2017-09-04T05:48:26.208Z


...

No match for {"type":"timeline","timeline":"29Sep16","name":[" ","Walking"," - 39 ft"],"location":[],"time":"2016-09-29T21:54:32.854Z","length":157941,"averageLat":33.48648296666667,"averageLon":-111.91830996666667}


2016-09-30T03:00:00.000Z
Unmatched 8 -  out of 1 - 6934 E Sandra Terrace, Psychological Counseling Services, Ralph H. Earle, PHD, North Michigan, Ralph H. Earle, PHD, Psychological Counseling Services, Mastro's City Hall, 6934 E Sandra Terrace
2016-09-28T13:58:00.000Z
2016-09-28T20:00:00.000Z
Unmatched 10 -  out of 2 - 6934 E Sandra Terrace, Psychological Counseling Services, Psychological Counseling Services, Scottsdale Art walk, Ralph H. Earle, PHD, McDonald's, McDonald's, Psychological Counseling Services, Psychological Counseling Services, 6934 E Sandra Terrace
2016-09-27T14:00:00.000Z
2016-09-28T02:00:00.000Z
Unmatched 7 -  out of 1 - 6934 E Sandra Terrace, Psychological Counseling Services, Psychological Counseling Services, Scottsdale Art walk, Ralph H. Earle, PHD, Psychological Counseling Services, 6934 E Sandra Terrace
2016-09-26T13:44:00.000Z


No match for {"type":"timeline","timeline":"26Sep16","name":[" ","Walking"," - 81 ft"],"location":[],"time":"2016-09-26T16:52:37.313Z","length":168767,"averageLat":33.486433,"averageLon":-111.9182503}
No match for {"type":"timeline","timeline":"26Sep16","name":[" ","Walking"," - 366 ft"],"location":[],"time":"2016-09-26T23:14:10.355Z","length":274645,"averageLat":33.48649906666667,"averageLon":-111.91842203333333}
No match for {"type":"timeline","timeline":"26Sep16","name":[" ","Walking"," - 169 ft"],"location":[],"time":"2016-09-26T23:51:35.935Z","length":600065,"averageLat":33.48642093333333,"averageLon":-111.9185675}


2016-09-26T15:00:00.000Z
Unmatched 6 -  out of 0 - 6934 E Sandra Terrace, Ralph H. Earle, PHD, Psychological Counseling Services, Ralph H. Earle, PHD, Psychological Counseling Services: Earle Marcus R PHD, 6934 E Sandra Terrace
2016-09-25T23:12:00.000Z
2016-09-25T17:00:00.000Z
Unmatched 6 -  out of 1 - Greenbrier East, Greenbrier East, LaserAway, LaserAway, Shawn M. Rodrigues, LPC, Shawn M. Rodrigues, LPC
2016-09-24T17:26:00.000Z
2016-09-24T16:30:00.000Z
Unmatched 1 -  out of 4 - Rock Springs Café
2016-09-23T15:56:00.000Z
2016-09-23T15:00:00.000Z
Unmatched 0 -  out of 6 - 
2016-09-23T02:27:00.000Z
2016-09-22T21:00:00.000Z
Unmatched 1 -  out of 3 - 6934 E Sandra Terrace


[ undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,